In [2]:
# Setup
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from tqdm import trange

%matplotlib inline
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# read in dataset with date column parsed
df = pd.read_csv('cleanedWeatherAUS.csv',
    parse_dates=['Date'],
    index_col='Date')


y = df['RainTomorrow']
xs = df[df.columns.difference(['RainTomorrow'])]

# split dataset, 80% train 20% test
from sklearn.model_selection import train_test_split
train_data_x, test_data_x, train_data_y, test_data_y = train_test_split(xs, y, train_size=0.8)

In [4]:
import torch.utils.data as data_utils
train_data = data_utils.TensorDataset(torch.Tensor(np.array(train_data_x)), torch.Tensor(np.array(train_data_y)))
test_data = data_utils.TensorDataset(torch.Tensor(np.array(test_data_x)), torch.Tensor(np.array(test_data_y)))

train_mean = [4.3997e+00, 4.5145e+00, 5.7088e+00, 4.7249e+01, 6.8562e+01, 2.4562e+01,
        2.3394e+01, 1.1880e+01, 1.0152e+03, 1.0178e+03, 1.5625e-01, 1.5844e+00,
        7.9468e+00, 2.2276e+01, 1.6602e+01, 8.0469e+00, 8.4531e+00, 8.9844e+00,
        3.9217e+01, 1.8432e+01, 1.4453e+01]
train_std = [ 2.2576,  2.1151,  2.2341, 17.3860, 19.0654, 15.3591,  6.0735,  5.6280,
         6.5641,  6.8042,  0.3660,  5.4969,  2.4661,  5.3701,  5.4730,  4.1113,
         4.8269,  4.5233, 13.8603, 10.3934, 10.9137]

transform = transforms.Compose([
   transforms.ToTensor(),
   transforms.Normalize((train_mean, ), (train_std, ))
])

train_data.transform = transform
test_data.transform = transform

batch_size = 64
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=True) 
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=True)

In [5]:
train_iter = iter(train_loader)
xs, labels = next(train_iter)

In [33]:
input_size = 21
hidden_size = 10 
output_size = 2 


import math 

class NNClassifier(torch.nn.Module):
    
    def __init__(self):
        super().__init__()

        # layer 1
        self.fc1 = torch.nn.Linear(input_size, hidden_size)

        # activation function
        self.act = torch.nn.Tanh()        
        
        # layer 2 
        self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
            
        # layer 3 
        self.fc3 = torch.nn.Linear(hidden_size, hidden_size)            
            
        # layer 4
        self.fc4 = torch.nn.Linear(hidden_size, output_size)
        
        # output
        self.log_softmax = torch.nn.Softmax(dim=1)

    
    def forward(self, x):
        # output after layer 1
        x = self.fc1(x)
        
        # apply activation function
        x = self.act(x)
        
        # output after layer 2
        x = self.fc2(x)
        
        # apply activation function
        x = self.act(x)
        
        # output after layer 3
        x = self.fc3(x)
        
        # apply activation function
        x = self.act(x)
        
        # output after layer 4
        x = self.fc4(x)
        
        # output after applying softmax
        return self.log_softmax(x)

model = NNClassifier().to(DEVICE)

# sanity check
print(model)

NNClassifier(
  (fc1): Linear(in_features=21, out_features=10, bias=True)
  (act): Tanh()
  (fc2): Linear(in_features=10, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=2, bias=True)
  (log_softmax): Softmax(dim=1)
)


In [34]:
def train_one_epoch(train_loader, model, device, optimizer, log_interval, epoch):
    model.train()
    losses = []
    counter = []
    cross_entropy_loss = torch.nn.CrossEntropyLoss()
    
    for i, (img, label) in enumerate(train_loader):
        img, label = img.to(device), label.to(device).long()
        
        # zero the gradients
        optimizer.zero_grad()
        # run forward on image data 
        outputs = model.forward(img)

        # compute loss
        loss = cross_entropy_loss(outputs, label)
        loss.backward()

        # adjust learning weights
        optimizer.step()
               
        # Record training loss every log_interval and keep counter of total training images seen
        if (i+1) % log_interval == 0:
            losses.append(loss.item())
            counter.append(
                (i * batch_size) + img.size(0) + epoch * len(train_loader.dataset))

    return losses, counter

In [35]:
def test_one_epoch(test_loader, train_loader, model, device):
    model.eval()
    test_loss = 0
    num_correct = 0
    train_num_correct = 0
    
    with torch.no_grad():
        for i, (features, label) in enumerate(test_loader):
            features, label = features.to(device), label.to(device)

            # ------------------
            # Write your implementation here.
            
            output = model.forward(features)
            pred = output.argmax(dim=1) 
            for iii in range(0, len(pred)):
                num_correct = num_correct + 1 if pred[iii] == label[iii] else num_correct + 0
                test_loss = test_loss + 0 if pred[iii] == label[iii] else test_loss + 1
        for i, (features, label) in enumerate(train_loader):
            features, label = features.to(device), label.to(device)

            # ------------------
            # Write your implementation here.
            
            output = model.forward(features)
            pred = output.argmax(dim=1) 
            for iii in range(0, len(pred)):
                train_num_correct = train_num_correct + 1 if pred[iii] == label[iii] else train_num_correct + 0
                # _loss = test_loss + 0 if pred[iii] == label[iii] else test_loss + 1

            # ------------------
            
    test_loss /= len(test_loader.dataset)
    return test_loss, num_correct, train_num_correct

In [36]:
#Hyperparameters
lr = 0.01
max_epochs=3
gamma = 0.95

# Recording data
log_interval = 100

# Instantiate optimizer (model was created in previous cell)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

train_losses = []
train_counter = []
test_losses = []
test_correct = []
for epoch in trange(max_epochs, leave=True, desc='Epochs'):
    train_loss, counter = train_one_epoch(train_loader, model, DEVICE, optimizer, log_interval, epoch)
    test_loss, num_correct, train_num_correct = test_one_epoch(test_loader, train_loader, model, DEVICE)

    # Record results
    train_losses.extend(train_loss)
    train_counter.extend(counter)
    test_losses.append(test_loss)
    test_correct.append(num_correct)
    print(f"Test accuracy: {num_correct/len(test_loader.dataset)}")
    
    print(f"Train counter: {train_num_correct/len(train_loader.dataset)}")

print(f"Test accuracy: {test_correct[-1]/len(test_loader.dataset)}")

Epochs:  33%|█████████████████████████▋                                                   | 1/3 [00:23<00:47, 23.92s/it]

Test accuracy: 0.7751679032314779
Train counter: 0.775981503947114


Epochs:  67%|███████████████████████████████████████████████████▎                         | 2/3 [00:52<00:26, 26.90s/it]

Test accuracy: 0.7751679032314779
Train counter: 0.775981503947114


Epochs: 100%|█████████████████████████████████████████████████████████████████████████████| 3/3 [01:20<00:00, 26.83s/it]

Test accuracy: 0.7751679032314779
Train counter: 0.775981503947114
Test accuracy: 0.7751679032314779


In [ ]:
# 1. Draw training loss curve
fig = plt.figure(figsize=(12,8))
plt.plot(train_counter, train_losses, label='Train loss')
plt.plot([i * len(train_loader.dataset) for i in range(1, max_epochs + 1)], 
         test_losses, label='Test loss', marker='o')
plt.xlim(left=0)
plt.ylim(bottom=0)
plt.title('Loss curve', fontsize=24)
plt.xlabel('Number of training examples seen', fontsize=16)
plt.ylabel('NLL', fontsize=16)
plt.legend(loc='upper right', fontsize=14)